This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

Note: Use a .env file and put your key in there and python-dotenv to access it in this notebook.

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a .env-example file in this repository to illusrtate what you need. Copy that to a file called .env and use your own api key in that .env file. Make sure you also have a .gitignore file with a line for .env added to it.

The standard Python gitignore is here you can just copy that.

In [1]:
# get api key from your env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the requests package, you are encouraged to not use any third party Python packages, such as pandas, and instead focus on what's available in the Python Standard Library (the collections module might come in handy: https://pymotw.com/3/collections/). Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

params = dict(start_date='2020-07-20', end_date='2020-07-20', api_key=API_KEY)
response = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=params)
response.status_code

200

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_data = response.json()
json_data.keys()

dict_keys(['dataset_data'])

In [5]:
json_data['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2020-07-20',
 'end_date': '2020-07-20',
 'frequency': 'daily',
 'data': [['2020-07-20',
   88.25,
   89.5,
   87.55,
   88.95,
   None,
   227635.0,
   20134893.65,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
#1 Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

year2017 = dict(start_date='2017-01-01', end_date='2017-12-31', api_key=API_KEY)
year2017_res = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=year2017)
year2017_res.status_code

200

In [7]:
#2 Convert the returned JSON object into a Python dictionary.

year2017_res_json = year2017_res.json()
year2017_res_json.keys()

dict_keys(['dataset_data'])

In [8]:
df = year2017_res_json['dataset_data']
# df

In [9]:
#3 Calculate what the highest and lowest opening prices were for the stock in this period.

daily_open = []
for i in range(len(df['data'])):
    daily_open.append(df['data'][i][1])
#     print('Daily Open for ' + str(df['data'][i][0]) + " = " + str(df['data'][i][1]))

In [10]:
highest = max([i for i in daily_open if i is not None])
lowest = min([i for i in daily_open if i is not None])

print("Highest opening price in 2017 = $" + str(highest))
print("Lowest opening price in 2017 = $" + str(lowest))

Highest opening price in 2017 = $53.11
Lowest opening price in 2017 = $34.0


In [11]:
#4 What was the largest change in any one day (based on High and Low price)?

daily_change = []
for i in range(len(df['data'])):
    daily_change.append(df['data'][i][2] - df['data'][i][3])
    
# print('Daily change for ' + str(df['data'][i][0]) + " = " + str(df['data'][i][2] - df['data'][i][3]))

In [12]:
print("Largest change in any one day based on High and Low Price = $" + str(max(daily_change)))

Largest change in any one day based on High and Low Price = $2.8100000000000023


In [13]:
#5 What was the largest change between any two days (based on Closing Price)?

closing_price = []
for i in range(len(df['data'])):
    closing_price.append(df['data'][i][4])
    i += 1

In [14]:
x = 0
closing_price_change = []
while x < len(df['data']) - 1:
    closing_price_change.append(closing_price[x+1] - closing_price[x])
    x+= 1

In [15]:
print("Largest change between any two days based on Closing Price = $" + str(max(closing_price_change)))

Largest change between any two days based on Closing Price = $2.559999999999995


In [16]:
#6 What was the average daily trading volume during this year?

trading_volume = []
for i in range(len(df['data'])):
    trading_volume.append(df['data'][i][6])
    i += 1

In [17]:
avg_trading_volume = sum(trading_volume) / len(trading_volume)
print("Average daily trading volume for 2017 = " + str(avg_trading_volume))

Average daily trading volume for 2017 = 89124.33725490196


In [18]:
# 7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Checks if odd or even number and calculates median accordingly
if len(trading_volume) % 2 > 0:
    trading_volume.sort()
    median_index = round(len(trading_volume) / 2)
    print("The median trading volume for 2017 = " + str(trading_volume[median_index]))
else:
    trading_volume.sort()
    median_index = round(len(trading_volume) / 2)
    lower = median_index - 1
    upper = median_index + 1
    median_volume = (trading_volume[lower] + trading_volume[upper]) / 2
    print("The median trading volume for 2017 = " + str(median_volume))

The median trading volume for 2017 = 76600.0
